In [ ]:
#!git clone https://github.com/dauparas/ProteinMPNN.git

In [ ]:
import torch
from torch import nn
from Prop3D.ml.datasets.DistributedProteinMPNNDataset import DistributedProteinMPNNDataset

import sys
sys.path.append("ProteinMPNN/training")
from model_utils import ProteinMPNN, featurize, get_std_opt

device = "cuda" if torch.cuda.is_available() else "cpu"

### Defined MPNN model
Instead of predicting 21 characters, only predict 3: is_electronegative, is_electropostive, is_neutral

### Define parameters

In [ ]:
hsds_endpoint = "http://uvarc.io"
cath_superfamily = "2/60/40/10" #Use / instead of .

#Could be charge, hydrophobicity, accessibility, 3 types of secondary structure, etc
predict_features = ["electrostatic_potential"] 

In [ ]:
dataset_train = DistributedProteinMPNNDataset(hsds_endpoint, cath_superfamily, cluster_level="S100")
training_loader = torch.utils.data.DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=10))
dataset_val = DistributedProteinMPNNDataset(hsds_endpoint, cath_superfamily, cluster_level="S100", validation=True)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=128, shuffle=False, num_workers=10))

In [ ]:
charge_to_idx = {(1,0,0):0, (0,1,0):1, (0,0,1):2}
def process_batch(batch):
    """Convert featuress into a new type of sequence with L=3
    """
    X, _, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
    S = [[charge_to_idx[(value==0,value<0,value>0)] for value in prot["feats"]] for prot in batch]
    S = torch.from_numpy(S, device=device)
    return X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all

model = ProteinMPNN(num_letters=3, vocab=3)
model = model.to(device)
optimizer = get_std_opt(model.parameters(), 128, 0)

In [ ]:
for epoch in range(30):
    for loader, is_train in [(training_loader, True), (val_loader, False)]:
        running_loss = 0
        for i, data in enumerate(loader):
            X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = process_batch(data)

            # Zero your gradients for every batch!
            optimizer.zero_grad()

            if is_train:
                # Make predictions for this batch
                mask_for_loss = mask*chain_M
                
                log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                _, loss_av_smoothed = loss_smoothed(S, log_probs, mask_for_loss)
                loss_av_smoothed.backward()
                
                loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)

                # Adjust learning weights
                optimizer.step()

                name = "TRAIN"

            else:
                # Make predictions for this batch
                outputs = model(inputs, labels=labels)
        
                # Compute the loss and its gradients
                loss = outputs.loss

                name = "VALIDATION"

            running_loss += loss
            if i%1000==0:
                last_loss = running_loss / 1000 # loss per batch
                print('  {} batch {} loss: {}'.format(name, i + 1, last_loss))
                running_loss = 0